<a href="https://colab.research.google.com/github/MADEENOH/ANLY699-Thesis-Trust-Signals/blob/main/Copy_of_LLM_Fine_Tuning_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cell 1

In [ ]:
!nvidia-smi

# Cell 2

In [ ]:
!pip install -q unsloth



# Cell 3

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


# cell 4

In [ ]:
# needed as this function doesn't like it when the lm_head has its size changed
from unsloth import tokenizer_utils

def do_nothing(*args, **kwargs):
    pass

tokenizer_utils.fix_untrained_tokens = do_nothing


# cell 5

In [ ]:
import torch
print(f"CUDA Capability: {torch.cuda.get_device_capability()}")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Number of classes (2 = binary classification: recommended or not)
NUM_CLASSES = 2

# Model setup
model_name = "unsloth/mistral-7b-bnb-4bit"
load_in_4bit = True
max_seq_length = 2048  # reduce if needed
dtype = None  # Let Unsloth auto-detect

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    load_in_4bit = load_in_4bit,
    max_seq_length = max_seq_length,
    dtype = dtype,
)


# Cell 6

In [ ]:
# Cell 6 (Corrected)

import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

input_path = "/content/data/Womens Clothing E-Commerce Reviews.csv"
data = pd.read_csv(input_path)

# Keep only needed columns and remove missing rows
data = data[['Review Text', 'Recommended IND']].dropna()
data = data.rename(columns={'Review Text': 'text', 'Recommended IND': 'label'})

# Use a sample as instructed
data = data.sample(n=5000, random_state=42)

# ✅ CHANGE: Map labels to text for more natural learning
data['label'] = data['label'].map({0: "Not Recommended", 1: "Recommended"})

# Check label distribution
print("Label distribution:")
print(data['label'].value_counts())
print("\n" + "="*30 + "\n")

# Train/validation split
train_df, val_df = train_test_split(data, test_size=0.1, random_state=42)
print("Train size:", len(train_df), "Val size:", len(val_df))
print("\nExample validation data:")
print(val_df.head())

# visuals EDA  

This block contains the code for the Recommendation Distribution and the Review Length Histogram.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set the style for our plots
sns.set_style("whitegrid")
sns.set_palette("viridis")

# Load the original data
input_path = "/content/data/Womens Clothing E-Commerce Reviews.csv"
original_data = pd.read_csv(input_path).dropna(subset=['Review Text', 'Recommended IND'])

# --- Visualization 1: Distribution of Recommendation Outcomes ---
plt.figure(figsize=(8, 6))
ax = sns.countplot(x='Recommended IND', data=original_data)
plt.title('Distribution of Recommendation Outcomes', fontsize=16)
plt.xlabel('Recommendation (0 = Not Recommended, 1 = Recommended)', fontsize=12)
plt.ylabel('Number of Reviews', fontsize=12)
ax.set_xticklabels(['Not Recommended', 'Recommended'])
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=11, color='black', xytext=(0, 5),
                textcoords='offset points')
plt.show()


# --- Visualization 2: Distribution of Review Text Lengths ---
original_data['review_length'] = original_data['Review Text'].str.len()
plt.figure(figsize=(12, 6))
sns.histplot(original_data['review_length'], bins=50, kde=True)
plt.title('Distribution of Review Text Lengths', fontsize=16)
plt.xlabel('Number of Characters in Review', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

# cell 7

In [ ]:
# Corrected Data Preparation Cell (Formatting + Tokenization)

from datasets import Dataset

# Define a clear, non-leaky prompt template for instruction-tuning
prompt_template = """Classify the sentiment of the following clothing review.
Answer with only 'Recommended' or 'Not Recommended'.

### Review:
{}

### Sentiment:
{}"""

# We need to add the EOS (End Of Sentence) token to the end of each example
# so the model knows when to stop generating.
EOS_TOKEN = tokenizer.eos_token

# 1. Formatting Function
def formatting_func(example):
    # This creates the full prompt with the answer for training
    full_text = prompt_template.format(example['text'], example['label']) + EOS_TOKEN
    return { "text": full_text }

# 2. Tokenizing Function
def tokenize_function(examples):
    # Tokenize the text, truncating sequences that are too long
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=max_seq_length, # This was defined in one of your first cells
    )

# Apply formatting to get a dataset with a 'text' column
formatted_train_dataset = Dataset.from_pandas(train_df).map(formatting_func, remove_columns=list(train_df.columns))
formatted_val_dataset = Dataset.from_pandas(val_df).map(formatting_func, remove_columns=list(val_df.columns))

# Apply tokenization to that dataset.
# `batched=True` makes this process much faster.
# `remove_columns=["text"]` deletes the now-unnecessary text column.
train_dataset = formatted_train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
val_dataset = formatted_val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

print("✅ Data preparation complete. The dataset now contains 'input_ids'.")
print("\nExample of a tokenized sample:")
print(train_dataset[0])

# CELL 8 PEFT

In [ ]:
# New Cell 2: PEFT Model Setup

# We don't need to change the lm_head.
# We target all linear layers for LoRA adaptation.
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

In [ ]:
# Diagnostic Cell
import transformers
import unsloth
import torch

print("--- Library Versions ---")
print(f"Unsloth version:       {unsloth.__version__}")
print(f"Transformers version:  {transformers.__version__}")
print(f"Torch version:         {torch.__version__}")
print("\n" + "="*50 + "\n")

print("--- TrainingArguments Documentation ---")
# This prints the official documentation for the TrainingArguments class
# that is active in your notebook environment.
from transformers import TrainingArguments
help(TrainingArguments)

# Cell 19 Define TrainingArguments & Initialize Trainer

In [ ]:
# Cell 19 (Final Solution - Safe Mode)

from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

print("Initializing Trainer in 'Safe Mode' to bypass the library bug.")
print("Evaluation arguments will be removed, and we will evaluate manually after training.")

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    # We remove eval_dataset from the Trainer for now.
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    args=TrainingArguments(
        output_dir="outputs",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        num_train_epochs=3, # You can set this back to 3
        learning_rate=5e-5,
        logging_steps=10,
        optim="adamw_8bit",
        bf16=True,
        fp16=False,
        seed=3407,
        # All problematic evaluation and saving strategy arguments are removed.
        # The model will save at the end of training.
        report_to="none",
    ),
)

print("✅ Trainer initialized successfully in safe mode.")

# Cell 20 Train the Model

In [ ]:
trainer.train()

# Training Loss Curve

This block creates the plot showing how your model's loss decreased during training.

In [ ]:
import matplotlib.pyplot as plt

# The trainer object holds the history of the training process
log_history = trainer.state.log_history

# Extract loss and steps from the log history
steps = []
losses = []
for log in log_history:
    if 'loss' in log:
        steps.append(log['step'])
        losses.append(log['loss'])

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(steps, losses, marker='o', linestyle='-', label="Training Loss")
plt.title('Model Training Loss Curve', fontsize=16)
plt.xlabel('Training Steps', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

# Cell 21 Inference

This part evaluates the model on the val set with batched inference

In [ ]:
# Cell 21 (Corrected)
from transformers import TextStreamer
import torch
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score

# Set up the prompt template for inference (the model fills in what's after "Sentiment:")
inference_template = """Classify the sentiment of the following clothing review.
Answer with only 'Recommended' or 'Not Recommended'.

### Review:
{}

### Sentiment:
"""

# Get the original texts and true labels from the validation dataframe
original_reviews = val_df['text'].tolist()
true_labels = val_df['label'].tolist()
predicted_labels = []

# Ensure model is in eval mode
model.eval()

print("Starting inference on the validation set...")
with torch.no_grad():
    for review in tqdm(original_reviews):
        # Format the prompt for inference
        prompt = inference_template.format(review)
        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

        # Generate the text output
        # max_new_tokens=5 is enough for "Recommended" or "Not Recommended"
        outputs = model.generate(**inputs, max_new_tokens=5, eos_token_id=tokenizer.eos_token_id)

        # Decode the generated part
        decoded_output = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]

        # Clean the output and store it
        cleaned_output = decoded_output.strip()
        predicted_labels.append(cleaned_output)

# --- Calculate Metrics ---
print("\nInference Complete. Calculating metrics...")

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"\nAccuracy: {accuracy:.4f}")

# Generate full classification report
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels, labels=["Recommended", "Not Recommended"]))

# Confusion Matrix (For a new cell after Cell 21)

This block visualizes the final performance of your model from the classification report.

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# This code assumes the 'true_labels' and 'predicted_labels' lists from your
# evaluation cell (Cell 21) are still in memory.

# Generate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels, labels=["Recommended", "Not Recommended"])

# Create a DataFrame for better labeling
cm_df = pd.DataFrame(cm,
                     index=['True: Recommended', 'True: Not Recommended'],
                     columns=['Pred: Recommended', 'Pred: Not Recommended'])

# Create the plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix of Model Performance', fontsize=16)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.show()

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Save the final, merged model for easy inference
# This combines the original Mistral model with your trained LoRA adapters
model.save_pretrained_merged("final_sentiment_model", tokenizer, save_method = "merged_16bit")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print("Loading the final, merged model from disk...")
# Load the final model that you saved locally
model_path = "final_sentiment_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# --- IMPORTANT ---
# Define the name for your new model repository on the Hub.
# You MUST replace 'your-hf-username' with your actual Hugging Face username.
repo_id = "MadeEnoh/mistral-7b-clothing-sentiment-v1"

print(f"\nUploading model to the Hugging Face Hub at: {repo_id}")
print("This may take several minutes as the model files are large...")

# Push both the model and the tokenizer to the Hub.
# `private=True` makes the model visible only to you. You can remove this or set it to False to make it public later.
model.push_to_hub(repo_id, private=True)
tokenizer.push_to_hub(repo_id, private=True)

print(f"✅ Success! Your model is now saved on the Hugging Face Hub.")
print(f"You can view it at: https://huggingface.co/{repo_id}")

#Attempt at Keyphrases

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

print("Starting key phrase analysis...")

# Load the dataset directly to ensure we have clean data
try:
    input_path = "/content/data/Womens Clothing E-Commerce Reviews.csv"
    original_data = pd.read_csv(input_path).dropna(subset=['Review Text', 'Recommended IND'])
except FileNotFoundError:
    print("Error: Dataset file not found. Please ensure the path is correct.")

# Function to get the top n-grams (phrases) from a body of text
def get_top_ngrams(corpus, n=None, ngram_range=(2, 2)):
    # Use CountVectorizer to count the frequency of 2-word phrases
    vec = CountVectorizer(ngram_range=ngram_range, stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# Separate the review text into two categories
recommended_corpus = original_data[original_data['Recommended IND'] == 1]['Review Text']
not_recommended_corpus = original_data[original_data['Recommended IND'] == 0]['Review Text']

# Get the top 15 most frequent phrases for each category
top_phrases_recommended = get_top_ngrams(recommended_corpus, n=15)
top_phrases_not_recommended = get_top_ngrams(not_recommended_corpus, n=15)

# Convert the results to a DataFrame for easy plotting
df_rec = pd.DataFrame(top_phrases_recommended, columns=['phrase', 'count'])
df_not_rec = pd.DataFrame(top_phrases_not_recommended, columns=['phrase', 'count'])

# --- Create the Visualization ---
print("Generating plot...")
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
sns.set_style("whitegrid")

# Plot for "Recommended" phrases
sns.barplot(x='count', y='phrase', data=df_rec, ax=axes[0], palette='Greens_d')
axes[0].set_title('Top 15 Key Phrases in "Recommended" Reviews', fontsize=16)
axes[0].set_xlabel('Frequency', fontsize=12)
axes[0].set_ylabel('Phrase (Trust Signal)', fontsize=12)

# Plot for "Not Recommended" phrases
sns.barplot(x='count', y='phrase', data=df_not_rec, ax=axes[1], palette='Reds_d')
axes[1].set_title('Top 15 Key Phrases in "Not Recommended" Reviews', fontsize=16)
axes[1].set_xlabel('Frequency', fontsize=12)
axes[1].set_ylabel('') # Hide redundant label

plt.suptitle('Discovered Trust Signals: An N-gram Analysis', fontsize=20, y=1.02)
plt.tight_layout()
plt.show()

print("\nAnalysis complete.")



### Story of the Charts


